# Практический кейс 2

## Анализ данных ритейл-магазина (магазин розничной торговли). Построение простой рекомендательной системы

### О задании

В этом задании я буду работать с данными о продажах товаров в магазинах розничной торговли и по предоставленным данным составлю рекомендации товаров с помощью простых статистических методов

Признаки данных торговой сети о покупках клиентов:
- `InvoiceNo` - номер чека. Товары с одинаковым `InvoiceNo` были приобретены одним покупателем в одной покупке.
- `StockCode` - универсальный идентификатор товара в базе данных магазина (один и тот же товар имеет единый `StockCode` во всех чеках)
- `Description` - название товара
- `Quantity` - количество товаров данного типа в чеке
- `UnitPrice` - цена одной единицы товара
- `InvoiceDate` - дата совершения покупки
- `CustomerID` - идентификационный номер пользователя (если покупка совершалась с каким-нибудь идентификатором, например с помощью скидочной карты магазина). Много покупок проходит без идентификации пользователя, поэтому в этой колонке может быть много пропусков.

### Базовые операции с датафреймом

Загрузим файл `online retail.csv` и выведем первые 5 строк

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/online retail.csv')

In [7]:
df.shape

(100000, 7)

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850.0
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 8:26,3.39,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 8:26,2.75,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850.0


**Первая часть работы**

Для выполнения первой части работы нам понадобятся только три столбца - `InvoiceNo`, `StockCode` и `Description`. Выделим их в отдельный датафрейм

In [9]:
df_first_part = df[['InvoiceNo', 'StockCode', 'Description']]

Нам будет удобнее работать с данными, если `id` товаров будут идти от 1 до числа уникальных товаров в датафрейме. Поэтому создадим новые удобные id товаров

Для этого сначала получим отсортированный список уникальных `StockCode` всех товаров

In [10]:
unique_sorted_stock_codes = sorted(df_first_part['StockCode'].unique())

Выясним сколько в датафрейме уникальных товаров. 

In [11]:
len(unique_sorted_stock_codes)

3128

А теперь сопоставим каждому элементу в полученном списке число по порядку от 0 до (количества уникальных элементов - 1) и сохраним это отображение в словарь

In [12]:
new_codes = dict(zip(unique_sorted_stock_codes, range(0, 3128)))
new_codes

{'10002': 0,
 '10120': 1,
 '10123C': 2,
 '10124A': 3,
 '10124G': 4,
 '10125': 5,
 '10133': 6,
 '10135': 7,
 '11001': 8,
 '15034': 9,
 '15036': 10,
 '15039': 11,
 '15044A': 12,
 '15044B': 13,
 '15044C': 14,
 '15044D': 15,
 '15056BL': 16,
 '15056N': 17,
 '15056P': 18,
 '15056bl': 19,
 '15056n': 20,
 '15056p': 21,
 '15058A': 22,
 '15058B': 23,
 '15058C': 24,
 '15060B': 25,
 '16008': 26,
 '16010': 27,
 '16011': 28,
 '16012': 29,
 '16014': 30,
 '16015': 31,
 '16016': 32,
 '16033': 33,
 '16045': 34,
 '16046': 35,
 '16048': 36,
 '16052': 37,
 '16054': 38,
 '16156L': 39,
 '16156S': 40,
 '16161M': 41,
 '16161P': 42,
 '16161U': 43,
 '16162L': 44,
 '16168M': 45,
 '16169K': 46,
 '16169M': 47,
 '16169N': 48,
 '16169P': 49,
 '16206B': 50,
 '16207A': 51,
 '16207B': 52,
 '16216': 53,
 '16218': 54,
 '16219': 55,
 '16225': 56,
 '16235': 57,
 '16236': 58,
 '16237': 59,
 '16238': 60,
 '16258A': 61,
 '17003': 62,
 '17007B': 63,
 '17011A': 64,
 '17011F': 65,
 '17012A': 66,
 '17012B': 67,
 '17012C': 68,
 '17

In [13]:
new_codes['71053']

2066

In [14]:
new_codes['85123A']

2679

### Для того, чтобы сделать рекомендации товаров с помощью статистических методов, нужно ответить на следующие вопросы о данных:
- какие пары товаров покупались вместе чаще всего?
- какие товары чаще всего покупались вместе с данным товаром?

Ответы на эти вопросы помогут в построении базовой системы рекомендаций товаров

### Построение матрицы смежности

Построим матрицу смежности размера `3128 * 3128`, в которой на пересечении строки `i` и столбца `j` будет стоять количество чеков, содержащих оба товара `i` и `j`. 

Чтобы было удобно навигироваться по матрице с помощью порядковых целочисленных индексов, и был создан словарь `new_codes` с отображением из старых индексов в новые индексы.

**Пример:** если товар со старым `StockCode=71053` (имеет новый индекс 2066) был совместно с товаром со `StockCode=85123A` (имеет новый индекс 2679) в 5 чеках, то в ячейке с индексами `[2066, 2679]` будет стоять цифра 5.

Реализуем вычисление этой матрицы. Для этого нужно:

0) Создать матрицу из нулей;

1) сгруппировать объекты по чекам;

внутри каждой группы:

2) сгенерировать попарные комбинации товаров внутри одного чека;

для каждой сгенерированной пары:

3) вычислить индексы `i` и `j` каждого из товаров в паре;

4) прибавить 1 к ячейке матрицы с индексами `[i,j]`.

**Примечание:** если два товара встречались комбинациях по 2 в разном порядке (напр. (item_1, item_2) и (item_2, item_1)), то единичку ставим в ячейку, у которой номер строки меньше (если `i` < `j`, то +1 к элементу `[i,j]`, иначе +1 к элементу `[j,i]`). Так мы добъемся того, что `матрица будет верхнетреугольной` и будет удобно считать суммарное количество покупок данного товара.

In [15]:
import itertools

In [16]:
import numpy as np
# создаем матрицу из нулей
item_pairs_counts = np.zeros((len(new_codes), len(new_codes)))

In [17]:
item_pairs_counts.shape

(3128, 3128)

In [18]:
for invoice_number, invoice_group in df_first_part.groupby('InvoiceNo'):
    for item_1, item_2 in itertools.combinations(
            list(invoice_group['StockCode']), 2):
        i, j = new_codes[item_1], new_codes[item_2]
        if i < j:
            item_pairs_counts[i, j] += 1
        else:
            item_pairs_counts[j, i] += 1

In [19]:
item_pairs_counts

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Тесты для проверки правильности вычисления матрицы

Напишем несколько тестов, чтобы проверить, правильно ли создалась матрица. Например, посчитаем по датафрейму, в скольких чеках встречалась пара товаров со `StockCode=85123A` и со `StockCode=71053`:

**Тест #1**

In [20]:
counter = 0
for invoice_num, invoice_group in df_first_part.groupby("InvoiceNo"):
    s = set(invoice_group['StockCode'])
    if '71053' in s and '37370' in s:
        counter += 1

print(counter)

24


Таким образом, пара товаров со `StockCode=85123A` и со `StockCode=71053` встречалась одновременно в 24 чеках.

Теперь получим это же число чеков, в которых эта пара товаров встречалась вместе с помощью вычисленной матрицы. Для этого нужно пойти в матрицу в ячейку с индексами `[i, j]`, где `i` и `j` - это новые присвоенные целочисленные индексы.

In [23]:
item_pairs_counts[new_codes['37370'], new_codes['71053']]

24.0

**Тест #2**

Напишем аналогичный тест для ещё одной пары товаров с другими `StockCode`

In [26]:
counter = 0
for invoice_num, invoice_group in df_first_part.groupby('InvoiceNo'):
    s = set(invoice_group['StockCode'])
    if '37370' in s and '22200' in s:
        counter += 1

print(counter)

8


In [27]:
item_pairs_counts[new_codes['22200'], new_codes['37370']]

8.0

Ок, кажется, что по крайней мере для двух пар вычисление значения числа чеков, в которых есть оба этих товара, выполнено правильно.

### Ответы на вопросы о данных

- Какие пары товаров покупались вместе чаще всего?

**Ответ:**

Сделаем созданную выше треугольную матрицу симметричной.

In [33]:
item_pairs_counts = item_pairs_counts + item_pairs_counts.T

In [34]:
# проверяем, что матрица стала симметричной
assert np.allclose(item_pairs_counts, item_pairs_counts.T)

In [36]:
index_in_flat_array = np.argmax(item_pairs_counts)
index_in_flat_array

4139668

In [37]:
index = np.unravel_index(index_in_flat_array, (3128, 3128))
index

(1323, 1324)

In [38]:
unique_sorted_stock_codes[index[0]]

'22469'

In [39]:
unique_sorted_stock_codes[index[1]]

'22470'

In [40]:
df_first_part[df_first_part['StockCode'] == '22469']['Description'].unique()[0] # совпадает со StockCode = '22470'

'HEART OF WICKER SMALL'

Что чаще всего покупали вместе с `KNITTED UNION FLAG HOT WATER BOTTLE` (`stock_code=84029G`)?


In [42]:
stock_code = '84029G'

In [43]:
item_index = new_codes[stock_code]
item_index

2216

In [44]:
complement_item_index = np.argmax(item_pairs_counts[item_index])
complement_item_index

2215

In [45]:
pair_count = np.max(item_pairs_counts[item_index])
pair_count

336.0

In [46]:
related_item_stoc_code = unique_sorted_stock_codes[complement_item_index]
related_item_stoc_code

'84029E'

In [50]:
complement_item = df_first_part[df_first_part['StockCode']
                                == unique_sorted_stock_codes[complement_item_index]]['Description'].unique()[0]
complement_item

'RED WOOLLY HOTTIE WHITE HEART.'

In [51]:
print('input index stock code:', stock_code)
print('input item index:', item_index)
print('complement item index:', complement_item_index)
print('pair count:', pair_count)
print('related item stock code:', related_item_stoc_code)
print('Complement Item:', complement_item)

complement_item

input index stock code: 84029G
input item index: 2216
complement item index: 2215
pair count: 336.0
related item stock code: 84029E
Complement Item: RED WOOLLY HOTTIE WHITE HEART.


'RED WOOLLY HOTTIE WHITE HEART.'

### Реализуем простую версию системы рекомендаций

**Бизнес-задача:** пусть пользователь набрал товаров в интернет-магазине и перешел в корзину. Порекомендуем ему еще что-нибудь подходящее.

**Решение:** будем к каждому целевому товару рекомендовать тот товар, который чаще всего покупали раньше с этим целевым товаром.

**Пример:** пусть в корзине лежат товары с `id` = `[10, 22, 31]`. По истории заказов посчитано, что с товаром с `id`=10 чаще всего покупали товар с `id`=71, с товаром с `id`=22 - товар с `id`=72, с товаром с `id`=31 - `id`=73.

Тогда список рекомендаций товаров к корзине `[10, 22, 31]` будут товары `[71, 72, 73]`

Реализуйем функцию, которая по входному списку товаров составляет список такой же длины, состоящий из товаров, которые наиболее часто покупаются вместе с каждым товаром из входного списка.


In [292]:
input_items_list = ['84406B', '22585', '20749']

In [295]:
def get_recommendation(input_items):
    complement_items = []
    for stock_code in input_items:
        
        item_index = new_codes[stock_code]
        complement_item_index = np.argmax(item_pairs_counts[item_index])
        pair_count = np.max(item_pairs_counts[item_index])
        related_item_stoc_code = unique_sorted_stock_codes[complement_item_index]
        complement_item = df_first_part[
            df_first_part['StockCode'] == unique_sorted_stock_codes[
                complement_item_index]]['Description'].unique()[0]
        
        complement_items.append(related_item_stoc_code)
        print('input index stock code:', stock_code)
        print('input item index:', item_index)
        print('complement item index:', complement_item_index)
        print('pair count:', pair_count)
        print('related item stock code:', related_item_stoc_code)
        print('Complement Item:', complement_item)
        print()
    return complement_items

In [296]:
get_recommendation(input_items_list)

input index stock code: 84406B
input item index: 2263
complement item index: 2679
pair count: 56.0
related item stock code: 85123A
Complement Item: WHITE HANGING HEART T-LIGHT HOLDER

input index stock code: 22585
input item index: 1433
complement item index: 1433
pair count: 52.0
related item stock code: 22585
Complement Item: PACK OF 6 BIRDY GIFT TAGS

input index stock code: 20749
input item index: 163
complement item index: 164
pair count: 28.0
related item stock code: 20750
Complement Item: RED RETROSPOT MINI CASES



['85123A', '22585', '20750']

### Подходы к улучшению

- Построенная выше система рекомендаций не учитывает персональной истории пользователей, а это могло бы быть полезной дополнительной информацией (id юзеров, кстати, есть в исходных данных).

Для того, чтобы рекомендации были более персональными можно высчитывать рекомендуемые товары ко всей истории покупок. И делать рекомендации на основе всей истории рекомендаций.

- Что, если один товар очень часто покупается со всеми товарами (5 литров воды/бананы/туалетная будмага) - тогда эти товары будут рекомендоваться неспецифично ко всем товарам. Как можно побороть влияние общеупотребимых (частых) товаров?

Можно ввести некоторый уменьшающий коэффициент, на который будут умножаться числа покупок этих товаров в паре с др. товарами.